<a href="https://colab.research.google.com/github/heewoonjeong/AI8_Study/blob/main/5_2%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D_%EA%B7%B8%EB%A6%AC%EB%93%9C_%EC%84%9C%EC%B9%98_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd      #검증세트가 필요한이유 일반화 성능을 올바르게 사용하려면 테스트 세트를 가능한 사용하면안됨 
wine = pd.read_csv('https://bit.ly/wine_csv_data')  # 테스트 세트를 사용하지않고 과소,인지 과대인지 측정하는 방법 >검증세트 테스트 세트 를 훈련시킴

In [ ]:
data = wine[['alcohol','sugar','pH']].to_numpy() 
target = wine['class'].to_numpy()                 # class 를 타깃으로 나머지열은 특성배열 저장 

In [ ]:
from sklearn.model_selection import train_test_split     # 테스트 세트의 검증 세트는 20~30%정도만 훈련 
train_input, test_input, train_target, test_target = train_test_split( # 테스트 세트 나눔 타깃 데이터를 train_input train_target 배열저장 
    data, target,test_size=0.2 , random_state =42
)

sub_input, val_input, sub_target, val_target = train_test_split(  # train_set을 train_test_split() 함수에 저장 sub,val에 저장 
    train_input, train_target, test_size= 0.2, random_state =42   
)

In [ ]:
print(sub_input.shape, val_input.shape) # 훈련세트와 검증세트 크기 확인 

(4157, 3) (1040, 3)


In [ ]:
from sklearn.tree import DecisionTreeClassifier 
dt= DecisionTreeClassifier()
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.8625


In [ ]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00906324, 0.00750828, 0.00709772, 0.00708508, 0.00680375]), 'score_time': array([0.00095344, 0.00085425, 0.00072455, 0.00076175, 0.00073695]), 'test_score': array([0.87211538, 0.85480769, 0.87584216, 0.8479307 , 0.83541867])}


In [ ]:
import numpy as np

print(np.mean(scores['test_score']))

0.857222921448138


In [ ]:
from sklearn.model_selection import StratifiedKFold 
scores = cross_validate(dt,train_input, train_target, cv= StratifiedKFold())
print(np.mean(scores['test_score']))

0.8579941881987118


In [ ]:
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state =42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8599214465688455


In [ ]:
from sklearn.model_selection import GridSearchCV
params = { 'min_impurity_decrease' : [ 0.0001,0.0002,0.0003,0.0004,0.0005]}

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params,n_jobs = -1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
dt= gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [ ]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [ ]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [ ]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [ ]:
params = {'min_impurity_decrease':np.arange(0.0001,0.001, 0.0001),
          'max_depth':range( 5, 20, 1),
          'min_samples_split': range(2, 100, 10)}

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state =42),params, n_jobs= -1)
gs.fit(train_input, train_target)


GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [ ]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [ ]:
params= {'min_impurity_decrease': uniform(0.0001,0.001),
         'max_depth':randint(20,50),
         'min_samples_split':randint(2, 25),
         'min_samples_leaf':randint(1,25),}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params,
                        n_iter= 100, n_jobs= -1, random_state= 42)
rs.fit(train_input,train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f65c60dadd0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f65c60da5d0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f65c4e14190>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f65c60dad90>},
                   random_state=42)

In [ ]:
print(rs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [ ]:
print(np.max(rs.cv_results_['mean_test_score']))

0.8695428296438884


In [ ]:
dt= rs.best_estimator_
print(dt.score(test_input, test_target))

0.86


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random' , random_state = 42), params,
                        n_iter= 100, n_jobs= -1, random_state= 42)
rs.fit(train_input,train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f65c60dadd0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f65c60da5d0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f65c4e14190>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f65c60dad90>},
                   random_state=42)

In [ ]:
dt= rs.best_estimator_
print(dt.score(test_input, test_target))

0.786923076923077
